In [1]:

import random
from pyomo.environ import *
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import time
import copy
import math



In [2]:
n= 18
p= 3
teams = list(range(1,n+1))
days = list(range(1,n))
days_dynamic = [x for x in days if x != 1]  # Removes all occurrences of 3
k = math.ceil((n - 1) / p)
rounds = [days_dynamic[i*p:((i+1)*p)] for i in range(k)]
rounds = list(filter(None, rounds))
print(rounds)

solution_keeper = pd.DataFrame(columns=['Day', 'Team 1', 'Team 2'])


[[2, 3, 4], [5, 6, 7], [8, 9, 10], [11, 12, 13], [14, 15, 16], [17]]


In [3]:
def first_model(teams,days):


    model = ConcreteModel()

    model.x = Var(teams, teams, days, within=Binary)

    # Constraints
    model.constraints = ConstraintList()


    for t in days:
        for i in teams:
            model.constraints.add(model.x[i,i,t] == 0)


    for i in teams:
        for j in teams:
            if i<j:
                model.constraints.add(sum(model.x[i,j,t] + model.x[j,i,t] for t in days) == 1)

    for i in teams:
        for t in days:
            model.constraints.add(sum(model.x[i,j,t] + model.x[j,i,t] for j in teams if j!=i ) == 1)

    solver = SolverFactory('gurobi')
    results = solver.solve(model, tee=True)

    
    # Print the results
    if results.solver.termination_condition == TerminationCondition.optimal:
        print("Optimal solution found.")
        for t in days:
            for i in teams:
                for j in teams:
                    if t ==1:
                        if value(model.x[i, j, t]) == 1:
                            print(f"Team {i} vs Team {j} in Slot {t}")
            

    else:
        print("No optimal solution found.")



    solution_df = pd.DataFrame(columns=['Day', 'Team 1', 'Team 2'])

    for t in days:
        for i in teams:
            for j in teams:
                if t==1:
                    if model.x[i, j, t].value == 1:
                        solution_df = pd.concat([solution_df, pd.DataFrame({'Day': [t], 'Team 1': [i], 'Team 2': [j]})],
                                            ignore_index=True)
    
  
    return solution_df
solution_df = first_model(teams, days)
print(solution_df)
solution_keeper = pd.concat([solution_keeper, solution_df])

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2023-12-14
Read LP format model from file C:\Users\ug033207\AppData\Local\Temp\tmpxg8hnsbx.pyomo.lp
Reading time = 0.02 seconds
x1: 765 rows, 5509 columns, 15912 nonzeros
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 765 rows, 5509 columns and 15912 nonzeros
Model fingerprint: 0x5cdc5369
Variable types: 1 continuous, 5508 integer (5508 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 306 rows and 2908 columns
Presolve time: 0.06s
Presolved: 459 rows, 2601 columns, 7803 nonzeros


In [4]:
result_df = pd.read_excel('C:/Users/ug033207/match_resultlar.xlsx', sheet_name= 'Germany first half')
result_df = pd.DataFrame(result_df)

old_ratings = {i: 0 for i in range(1, n+1)}
U_ij = np.zeros((n, n)) 
ratings_array = np.array(list(old_ratings.values()))
for i in range(n):
    for j in range(n):
        U_ij[i, j] = abs((ratings_array[i] - ratings_array[j]))

output_ratings = {index: value for index, value in enumerate(ratings_array, start=1)}
rating_df = pd.DataFrame(U_ij, columns=teams, index=teams)

In [5]:


def first_round_rating(ratings, solution_df, result_df):

    relevant_matches = pd.DataFrame(columns=['Team 1', 'Team 2', 'Result'])

    for _, row in solution_df.iterrows():
        team1 = row['Team 1']
        team2 = row['Team 2']
        day = row['Day']
        
        for _, row in result_df.iterrows():
            team11 = row ['Team 1']
            team22 = row ['Team 2']
            result = row ['Result']
            if day == 1:

                if team1 == team11 and team2 == team22 or team1 == team22 and team2 == team11 :
                    relevant_matches = pd.concat([relevant_matches, pd.DataFrame({'Team 1': [team1], 'Team 2': [team2], 'Result': [result]})],
                                                    ignore_index=True)
    relevant_matches = relevant_matches.drop_duplicates()           
    for _, row in relevant_matches.iterrows():
            team1 = row['Team 1']
            team2 = row['Team 2']
            winner = row['Result']

            if winner == team1:
                ratings[team1] += 3
            elif winner == team2:
                ratings[team2] += 3
            else:
                ratings[team1] += 1
                ratings[team2] += 1
                    
    ratings_array = np.array(list(ratings.values()))
    

    U_ij = np.zeros((n, n)) 

    for i in range(n):
        for j in range(n):
            U_ij[i, j] = abs((ratings_array[i] - ratings_array[j]))
    

    output_ratings = {index: value for index, value in enumerate(ratings_array, start=1)}
    ratings_df = pd.DataFrame(U_ij, columns=teams, index=teams)

    return output_ratings, ratings_df

ratings, ratings_df = first_round_rating(output_ratings, solution_df, result_df)

In [6]:

def dynamic_model(teams,days, rating_df, solution_keeper, r):


    # Create ConcreteModel
    model = ConcreteModel()
    # Variables
    model.x = Var(teams, teams, days, within=Binary)
    model.u = Param(teams, teams, within=NonNegativeReals, mutable=True)



    # Constraints
    model.constraints = ConstraintList()


    for t in days:
        for i in teams:
            model.constraints.add(model.x[i,i,t] == 0)


    for i in teams:
        for j in teams:
            if i<j:
                model.constraints.add(sum((model.x[i,j,t] + model.x[j,i,t]) for t in days) == 1)

    for i in teams:
        for t in days:
            model.constraints.add(sum((model.x[i,j,t] + model.x[j,i,t]) for j in teams if j!=i ) == 1)


    # Read data from the file and assign it to model.u
    for i in teams:
        for j in teams:
            model.u[i, j] = rating_df.iloc[i - 1, j - 1]

    # Fiy variables based on the provided solution schedule
    previous_round = []
    for _, row in solution_keeper.iterrows():
        day = row['Day']
        team1 = row['Team 1']
        team2 = row['Team 2']
        model.x[team1, team2, day].fix(1)
        previous_round.append(day)


    def rule_of(model):
        return (sum(sum(sum((model.u[i, j] * model.x[i, j, t]) for t in r) for j in teams)for i in teams))


    model.obj = Objective(rule=rule_of, sense=minimize)

    
    # Solve the model
    solver = SolverFactory('gurobi')
    results= solver.solve(model)


    solution_df = pd.DataFrame(columns=['Day', 'Team 1', 'Team 2'])
    for t in r:
        for i in teams:
            for j in teams:
                if model.x[i, j, t].value == 1:
                    new_row = pd.DataFrame({'Day': [t], 'Team 1': [i], 'Team 2': [j]})
                    solution_df = pd.concat([solution_df, new_row]).reset_index(drop=True)

    if results.solver.termination_condition == TerminationCondition.optimal:
        print("Optimal solution found.")
        for t in r:
            for i in teams:
                for j in teams:
                    if value(model.x[i, j, t]) == 1:
                        print(f"Team {i} vs Team {j} in Slot {t}")
        

    else:
        print("No optimal solution found.")


    return solution_df
    
    

def rating(ratings, teams, solution_df, result_df, r):



    relevant_matches = pd.DataFrame(columns=['Team 1', 'Team 2', 'Result'])


    for _, row in solution_df.iterrows():
        team1 = row['Team 1']
        team2 = row['Team 2']
        day = row['Day']
        
        for _, row in result_df.iterrows():
            team11 = row ['Team 1']
            team22 = row ['Team 2']
            result = row ['Result']
            for i in r:
                if i==day:
                    if team1 == team11 and team2 == team22 or team1 == team22 and team2 == team11 :
                        relevant_matches = pd.concat([relevant_matches, pd.DataFrame({'Team 1': [team1], 'Team 2': [team2], 'Result': [result]})],
                                                    ignore_index=True)
    relevant_matches = relevant_matches.drop_duplicates()           
    print(relevant_matches)
    # Iterate over each row in the DataFrame
    for _, row in relevant_matches.iterrows():
        team1 = row['Team 1']
        team2 = row['Team 2']
        winner = row['Result']

        if winner == team1:
            ratings[team1] += 3
        elif winner == team2:
            ratings[team2] += 3
        else:
            ratings[team1] += 1
            ratings[team2] += 1
                
    ratings_array = np.array(list(ratings.values()))


    U_ij = np.zeros((n, n)) 

    for i in range(n):
        for j in range(n):
            U_ij[i, j] = abs((ratings_array[i] - ratings_array[j]))
    

    output_ratings = {index: value for index, value in enumerate(ratings_array, start=1)}

    ratings_df = pd.DataFrame(U_ij, columns=teams, index=teams)
    print(output_ratings)
    return ratings, ratings_df


for r in rounds:
    solution_df = dynamic_model(teams, days, rating_df, solution_keeper, r)
    solution_keeper = pd.concat([solution_keeper, solution_df])
    ratings, rating_df = rating(ratings, teams, solution_df, result_df, r)

Optimal solution found.
Team 2 vs Team 4 in Slot 2
Team 3 vs Team 1 in Slot 2
Team 5 vs Team 7 in Slot 2
Team 6 vs Team 8 in Slot 2
Team 9 vs Team 13 in Slot 2
Team 10 vs Team 11 in Slot 2
Team 15 vs Team 12 in Slot 2
Team 16 vs Team 18 in Slot 2
Team 17 vs Team 14 in Slot 2
Team 1 vs Team 4 in Slot 3
Team 3 vs Team 2 in Slot 3
Team 7 vs Team 6 in Slot 3
Team 8 vs Team 5 in Slot 3
Team 9 vs Team 14 in Slot 3
Team 11 vs Team 18 in Slot 3
Team 13 vs Team 10 in Slot 3
Team 15 vs Team 17 in Slot 3
Team 16 vs Team 12 in Slot 3
Team 4 vs Team 8 in Slot 4
Team 5 vs Team 1 in Slot 4
Team 6 vs Team 2 in Slot 4
Team 7 vs Team 3 in Slot 4
Team 10 vs Team 14 in Slot 4
Team 13 vs Team 11 in Slot 4
Team 15 vs Team 9 in Slot 4
Team 17 vs Team 16 in Slot 4
Team 18 vs Team 12 in Slot 4
   Team 1 Team 2 Result
0       2      4      2
1       3      1    tie
2       5      7    tie
3       6      8      6
4       9     13      9
5      10     11     10
6      15     12     12
7      16     18     16
8   

In [7]:
result_df2 = pd.read_excel('C:/Users/ug033207/match_resultlar.xlsx', sheet_name= 'Germany second half')
result_df2 = pd.DataFrame(result_df2)

ikinci_yari_first_week = first_model(teams, days)
print(ikinci_yari_first_week)
ratings, rating_df = first_round_rating(ratings, ikinci_yari_first_week, result_df2)

solution_keeper2 = pd.DataFrame(columns=['Day', 'Team 1', 'Team 2'])
solution_keeper2 = pd.concat([solution_keeper2, ikinci_yari_first_week])
print(solution_keeper2)
print(rounds)
for r in rounds:
    solution_df2 = dynamic_model(teams, days, rating_df, solution_keeper2, r)
    solution_keeper2 = pd.concat([solution_keeper2, solution_df2])
    ratings, rating_df = rating(ratings, teams, solution_df2, result_df2, r)

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2023-12-14
Read LP format model from file C:\Users\ug033207\AppData\Local\Temp\tmplmrml_l7.pyomo.lp
Reading time = 0.04 seconds
x1: 765 rows, 5509 columns, 15912 nonzeros
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 765 rows, 5509 columns and 15912 nonzeros
Model fingerprint: 0x5cdc5369
Variable types: 1 continuous, 5508 integer (5508 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 306 rows and 2908 columns
Presolve time: 0.02s
Presolved: 459 rows, 2601 columns, 7803 nonzeros
